<a href="https://colab.research.google.com/github/mdpw/msc-ir-code/blob/main/Session%2012/4B_Problem_specific_Email_Response_Generator_Practical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Cell 1: Install dependencies and Ollama
import subprocess
import time
import os

# Install zstd first
print("📦 Installing zstd...")
!sudo apt-get update -qq
!sudo apt-get install -y zstd

# Now install Ollama
print("📦 Installing Ollama...")
!curl -fsSL https://ollama.com/install.sh | sh

print("✅ Ollama installed successfully!")

📦 Installing zstd...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  zstd
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 603 kB of archives.
After this operation, 1,695 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 zstd amd64 1.4.8+dfsg-3build1 [603 kB]
Fetched 603 kB in 0s (2,356 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a con

In [9]:
# Cell 2: Start Ollama server
import subprocess
import time

# Start Ollama server in the background
print("🚀 Starting Ollama server...")
process = subprocess.Popen(
    ["ollama", "serve"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

# Wait for server to start
time.sleep(10)

print("✅ Ollama server is running!")

🚀 Starting Ollama server...
✅ Ollama server is running!


In [10]:
# Cell 3: Pull the model
print("📥 Downloading phi3:mini model (this takes 2-3 minutes)...")
!ollama pull phi3:mini
print("✅ Model downloaded!")

📥 Downloading phi3:mini model (this takes 2-3 minutes)...

✅ Model downloaded!


In [11]:
# Cell 4: Install Python packages
!pip install ollama streamlit nest_asyncio pyngrok -q
print("✅ Python packages installed!")

✅ Python packages installed!


In [12]:
# Cell 5: Test the setup
import ollama

# Quick test
response = ollama.chat(
    model="phi3:mini",
    messages=[{"role": "user", "content": "Say hello!"}]
)
print("Test response:", response['message']['content'])

Test response: Hello there! It's nice to meet you. How can I assist you today?


In [13]:
import nest_asyncio

In [14]:
# Hey Jupyter kernel, behave nicely if someone tries to start another async loop inside you.
# manage clashes of the kernal, when streamlit is running
# event loop management
nest_asyncio.apply()

In [15]:
def analyze_customer_email(email: str) -> str:
    """
    Step 1: Analyze and extract key info (intent, order number, sentiment, etc.)
    """
    extract_prompt = f"""
    Analyze the following customer email.
    1. Classify the intent (refund request, delivery status, product inquiry, complaint, etc.)
    2. Extract key details such as order number, product, sentiment, urgency.
    3. Return the result as JSON with keys: intent, order_number, product, sentiment, urgency, summary.

    Customer Email:
    {email}
    """
    response = ollama.chat(
        model="phi3:mini",
        messages=[{"role": "user", "content": extract_prompt}]
    )
    return response['message']['content']

In [16]:
def draft_reply(email: str, analysis: str) -> str:
    """
    Step 2: Generate support reply with empathy, professionalism, and business alignment
    """
    business_context = """
    You are a professional customer support representative at a consumer electronics company.
    Always:
    - Be polite, empathetic, and concise.
    - Acknowledge the customer situation.
    - Include relevant details (order info, delivery timing, etc.) when available.
    - End positively, reassuring the customer that we value them.
    """

    reply_prompt = f"""
    {business_context}

    The customer sent the following email:
    {email}

    Analysis of their message:
    {analysis}

    Draft a polite, appropriate email response:
    """
# "role": "user" means: treat this as if it came from the “user” in a chat.
    response = ollama.chat(
        model="phi3:mini",
        messages=[{"role": "user", "content": reply_prompt}]
    )
    return response['message']['content']
# =====================================
# 4️⃣ Test the pipeline directly (without UI)
# =====================================
customer_email = """
Hi Support,
I ordered a Bluetooth speaker last week (Order #12345), but it still hasn’t arrived.
Can you please tell me when it will be delivered? I needed it for a birthday gift.
"""

analysis = analyze_customer_email(customer_email)
reply = draft_reply(customer_email, analysis)

print("🔍 Extracted Understanding:\n", analysis, "\n")
print("✉️ Draft Reply:\n", reply)

🔍 Extracted Understanding:
 ```json

{

  "intent": "delivery status inquiry",

  "order_number": "12345",
  "product": "Bluetooth speaker",

  "sentiment": "frustrated/anxious due to urgency and importance of the item for a birthday gift",

  "urgency": "high - as it is needed by tomorrow (implied)",

  "summary": "Customer inquires about the delayed delivery status of a Bluetooth speaker ordered last week, expressing frustration because it's intended as a timely birthday gift. Provides order number and seeks immediate information."

}

``` 

✉️ Draft Reply:
 
Dear [Customer Name],


I hope this message finds you well. Thank you for reaching out to us regarding your recent purchase of our Bluetooth speaker (Order #12345). I understand how important it is for the gift delivery timings, and we sincerely apologize for any inconvenience caused by its delay. 


Upon reviewing your order status immediately after receiving this message, I've noticed that there appears to have been a slight l

In [17]:
%%writefile support_ai_app.py
import streamlit as st
import ollama

def analyze_customer_email(email: str) -> str:
    extract_prompt = f"""
    Analyze the following customer email.
    1. Classify the intent (refund request, delivery status, product inquiry, complaint, etc.)
    2. Extract key details such as order number, product, sentiment, urgency.
    3. Return the result as JSON with keys: intent, order_number, product, sentiment, urgency, summary.

    Customer Email:
    {email}
    """
    response = ollama.chat(
        model="phi3:mini",
        messages=[{"role": "user", "content": extract_prompt}]
    )
    return response['message']['content']


def draft_reply(email: str, analysis: str) -> str:
    business_context = """
    You are a professional customer support representative at a consumer electronics company.
    Always:
    - Be polite, empathetic, and concise.
    - Acknowledge the customer situation.
    - Include relevant details (order info, delivery timing, etc.) when available.
    - End positively, reassuring the customer that we value them.
    """

    reply_prompt = f"""
    {business_context}

    The customer sent the following email:
    {email}

    Analysis of their message:
    {analysis}

    Draft a polite, appropriate email response:
    """

    response = ollama.chat(
        model="phi3:mini",
        messages=[{"role": "user", "content": reply_prompt}]
    )
    return response['message']['content']


# ------------------ UI -------------------------
st.set_page_config(page_title="AI Customer Support Assistant", page_icon="📩")

st.title("📩 AI-Powered Customer Support Assistant")
st.write("Paste a customer email and let AI analyze and draft a polite, professional reply.")

# Input from user
customer_email = st.text_area("Customer Email:", height=200, placeholder="Paste the customer email here...")

if st.button("Generate Reply"):
    if customer_email.strip():
        with st.spinner("🔍 Analyzing email..."):
            analysis = analyze_customer_email(customer_email)
        st.subheader("Extracted Understanding")
        st.json(analysis)

        with st.spinner("✍️ Drafting reply..."):
            reply = draft_reply(customer_email, analysis)

        st.subheader("AI Drafted Reply")
        st.write(reply)
    else:
        st.warning("Please paste a customer email first.")

Writing support_ai_app.py


In [18]:
!streamlit run support_ai_app.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.55.132.98:8501

  Stopping...


In [20]:
!streamlit run support_ai_app.py --server.headless true --server.port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.55.132.98:8501

  Stopping...
